In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib
import os

AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.keras.backend.clear_session()  # For easy reset of notebook state.

tf.__version__
tf.executing_eagerly()


True

In [2]:

ds = tfds.load(name="imagenette", with_info=True, split=["train[:1000]","validation[:100]"])
ds_train=ds[0][0]
ds_test=ds[0][1]
assert isinstance(ds_train, tf.data.Dataset)

def normalize(x):
    y = {'image': tf.image.convert_image_dtype(x['image'], tf.float32), 'label': x['label']}
    y = (tf.image.resize(y['image'], (224,224)), y['label'])
    return y
    
ds_train = ds_train.map(lambda x: normalize(x))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(100)
ds_train = ds_train.batch(4)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(4)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [72]:
class ResNetBlock(tf.keras.layers.Layer):
    def __init__(self, filters=64, stride=1):
        super(ResNetBlock, self).__init__()
        self.stride = stride
        self.conv1 = layers.Conv2D(filters, 3, strides=(self.stride, self.stride), activation='relu', padding='same')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()
        if stride == 2:
            self.conv3 = layers.Conv2D(filters,1, strides=(self.stride, self.stride), padding='same' )
            self.bn3 = layers.BatchNormalization()
        self.add1 = layers.Add()
    
    def call(self, inputs, training=False):
        print('inputs', inputs)
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        print('conv1', x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        print('conv2', x)
        if self.stride == 2:
            inputs = self.conv3(x)
            inputs = self.bn3(x)
            print('conv3', inputs)
        return tf.nn.relu(self.add1([x, inputs]))
    

class CustomResNetModel(tf.keras.Model):
    def __init__(self):
        super(CustomResNetModel, self).__init__()
        self.conv1 = layers.Conv2D(64, 7, strides=(2, 2), padding='same')
        self.pool1 = layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')
        self.res_block1 = ResNetBlock(64)
        self.res_block2 = ResNetBlock(64)
        self.res_block3 = ResNetBlock(64)
        self.res_block4 = ResNetBlock(128, 2)
        self.res_block5 = ResNetBlock(128)
        self.res_block6 = ResNetBlock(128)
        self.res_block7 = ResNetBlock(128)
        self.res_block8 = ResNetBlock(256, 2)
        self.res_block9 = ResNetBlock(256)
        self.res_block10 = ResNetBlock(256)
        self.res_block11 = ResNetBlock(256)
        self.res_block12 = ResNetBlock(256)
        self.res_block13 = ResNetBlock(256)
        self.res_block14 = ResNetBlock(512, 2)
        self.res_block15 = ResNetBlock(512)
        self.res_block16 = ResNetBlock(512)
        self.pool2 = layers.GlobalAveragePooling2D()
        self.dense1 = layers.Dense(1000, activation='relu')
        self.dense2 = layers.Dense(100, activation='softmax')

    def call(self,inputs):
        #print('inputs',inputs)
        x = self.conv1(inputs)
        #print('after conv',x)
        x = self.pool1(x)
        #print('after pool',x)
        x = self.res_block1(x)
        #print('after resblock 1',x)
        x = self.res_block2(x)
        #print('after resblock 2',x)
        x = self.res_block3(x)
        #print('after resblock 3',x)
        x = self.res_block4(x)
        #print('after resblock 4',x)
        x = self.res_block5(x)
        x = self.res_block6(x)
        x = self.res_block7(x)
        x = self.res_block8(x)
        x = self.res_block9(x)
        x = self.res_block10(x)
        x = self.res_block11(x)
        x = self.res_block12(x)
        x = self.res_block13(x)
        x = self.res_block14(x)
        x = self.res_block15(x)
        x = self.res_block16(x)
        x = self.pool2(x)
        x = self.dense1(x)
        
        return self.dense2(x)

In [73]:
model = CustomResNetModel()

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)


In [74]:
model.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=15,
    validation_data=ds_test,
)
model.run_eagerly = True

inputs Tensor("custom_res_net_model_23/max_pooling2d_23/Identity:0", shape=(None, 56, 56, 64), dtype=float32)
conv1 Tensor("custom_res_net_model_23/res_net_block_296/batch_normalization_599/Identity:0", shape=(None, 56, 56, 64), dtype=float32)
conv2 Tensor("custom_res_net_model_23/res_net_block_296/batch_normalization_600/Identity:0", shape=(None, 56, 56, 64), dtype=float32)
inputs Tensor("custom_res_net_model_23/res_net_block_296/Identity:0", shape=(None, 56, 56, 64), dtype=float32)
conv1 Tensor("custom_res_net_model_23/res_net_block_297/batch_normalization_601/Identity:0", shape=(None, 56, 56, 64), dtype=float32)
conv2 Tensor("custom_res_net_model_23/res_net_block_297/batch_normalization_602/Identity:0", shape=(None, 56, 56, 64), dtype=float32)
inputs Tensor("custom_res_net_model_23/res_net_block_297/Identity:0", shape=(None, 56, 56, 64), dtype=float32)
conv1 Tensor("custom_res_net_model_23/res_net_block_298/batch_normalization_603/Identity:0", shape=(None, 56, 56, 64), dtype=float32

conv2 Tensor("custom_res_net_model_23/res_net_block_301/batch_normalization_611/FusedBatchNormV3:0", shape=(None, 28, 28, 128), dtype=float32)
inputs Tensor("custom_res_net_model_23/res_net_block_301/Relu:0", shape=(None, 28, 28, 128), dtype=float32)
conv1 Tensor("custom_res_net_model_23/res_net_block_302/batch_normalization_612/FusedBatchNormV3:0", shape=(None, 28, 28, 128), dtype=float32)
conv2 Tensor("custom_res_net_model_23/res_net_block_302/batch_normalization_613/FusedBatchNormV3:0", shape=(None, 28, 28, 128), dtype=float32)
inputs Tensor("custom_res_net_model_23/res_net_block_302/Relu:0", shape=(None, 28, 28, 128), dtype=float32)
conv1 Tensor("custom_res_net_model_23/res_net_block_303/batch_normalization_614/FusedBatchNormV3:0", shape=(None, 14, 14, 256), dtype=float32)
conv2 Tensor("custom_res_net_model_23/res_net_block_303/batch_normalization_615/FusedBatchNormV3:0", shape=(None, 14, 14, 256), dtype=float32)
conv3 Tensor("custom_res_net_model_23/res_net_block_303/batch_normali

inputs Tensor("custom_res_net_model_23/max_pooling2d_23/MaxPool:0", shape=(None, 56, 56, 64), dtype=float32)
conv1 Tensor("custom_res_net_model_23/res_net_block_296/batch_normalization_599/FusedBatchNormV3:0", shape=(None, 56, 56, 64), dtype=float32)
conv2 Tensor("custom_res_net_model_23/res_net_block_296/batch_normalization_600/FusedBatchNormV3:0", shape=(None, 56, 56, 64), dtype=float32)
inputs Tensor("custom_res_net_model_23/res_net_block_296/Relu:0", shape=(None, 56, 56, 64), dtype=float32)
conv1 Tensor("custom_res_net_model_23/res_net_block_297/batch_normalization_601/FusedBatchNormV3:0", shape=(None, 56, 56, 64), dtype=float32)
conv2 Tensor("custom_res_net_model_23/res_net_block_297/batch_normalization_602/FusedBatchNormV3:0", shape=(None, 56, 56, 64), dtype=float32)
inputs Tensor("custom_res_net_model_23/res_net_block_297/Relu:0", shape=(None, 56, 56, 64), dtype=float32)
conv1 Tensor("custom_res_net_model_23/res_net_block_298/batch_normalization_603/FusedBatchNormV3:0", shape=(N

     67/Unknown - 98s 1s/step - loss: 3.4019 - accuracy: 0.1212

KeyboardInterrupt: 